In [1]:
from scipy.integrate import quad
import numpy as np

---

1. Вычислить приближенное значение интеграла с помощью: $\\$
a) квадратурной формулы Гаусса с шестью узлами. $\\$
б) формулы Симпсона с использованием алгоритма автоматического выбора шага интегрирования до достижения погрешности eps=10-5 и уточнить полученное значение, используя интерполяцию по Ричардсону; 
2. Ответить на контрольные вопросы.

Задания представлены в виде встроенной функции:

NIntegrate [ f[x], {x, a., b.} ],

которая вычисляет приближенное значение интеграла $ \int_{a}^{b} f(x)dx $.

В задании указан также результат ее применения.

Формула Симпсона

NIntegrate[ Exp [x] Sin [x]  , {x, 0., N[Pi]} ]

12.0703

---

### Постановка задачи
<u>Цель</u>: вычислить приближённое значение интеграла методами численного интегрирования, используя формулу Гаусса с шестью узлами и формулу Симпсона с использованием алгоритма автоматического выбора шага интегрирования до достижения заданной погрешности. Уточнить полученное значение, используя интерполяцию по Ричардсону.

<u>Исходные данные</u>: Заданная функция f(x), интервал интегрирования, заданная точность $ \epsilon $, узлы и коэффициенты для приближённого вычисления интеграла.

<u>Модельные представления</u>: Квадратурная формула Гаусса с шестью узлами. Метод Симпсона. Алгоритм автоматического выбора шага интегрирования. Интерполяция по Ричардсону.

<u>Критерий оценки результата</u>: Совпадение значений интегралов, вычисленных с использованием квадратурной формулы Гаусса, метода Симпсона и с помощью встроенной функции NIntegrate.

---

In [2]:
# Заданный интервал интегрирования
a, b = 0, np.pi

# Заданная погрешность
eps = 10e-5

# Заданная функция
def f(x):
    return np.exp(x) * np.sin(x)

# Используем функцию quad для расчета точного значения интеграла
exact_integral, _ = quad(f, a, b)

print(exact_integral)

12.070346316389633


#### Квадратурная формула Гаусса

Для приближенного вычисления интеграла на отрезке [-1, 1] используются формула (в которой коэффициенты и узлы известны из специальных таблиц):

$$ I = \int_{-1}^1 f(x)dx \approx \sum_{i=1}^n A_i f(x_i) $$

Для вычисления интеграла на произвольном отрезке [a, b] следует выполнить замену переменных:

$$ x = \frac{b - a}{2} t + \frac{a + b}{2} $$

Тогда

$$ dx = \frac{b - a}{2}  dt $$

$$
I = \int_a^b f(x) dx = \frac{b - a}{2} \int_{-1}^1 f \left( \frac{b - a}{2} t + \frac{a + b}{2} \right) dt \approx \frac{b - a}{2} \sum_{i=1}^{n} A_i f \left( \frac{b - a}{2} t_i + \frac{a + b}{2} \right)
$$

$$
\int_a^b f(x) dx \approx \frac{b - a}{2} \sum_{i=1}^n A_i f \left( \frac{b - a}{2} t_i + \frac{a + b}{2} \right)
$$

In [3]:
# Узлы и веса квадратурной формулы Гаусса с 6 узлами
A = np.array([0.1713244924, 0.3607615730, 0.4679139346,
                    0.4679139346, 0.3607615730, 0.1713244924])
t = np.array([-0.9324695142, -0.6612093864, -0.2386191861, 
                  0.2386191861, 0.6612093864, 0.9324695142])

In [4]:
# Квадратурная функция Гаусса
def gauss_quadrature(f, a, b, A, t):
    result = 0.0

    for i in range(len(t)):
        result += A[i] * f((b - a) / 2 * t[i] + (a + b) / 2)

    result *= (b - a) / 2

    return result


In [5]:
gauss_integral = gauss_quadrature(f, a, b, A, t)

print(gauss_integral)

12.07034648427794


---

#### Формула Симпсона

Пусть $ h = \frac{b - a}{2m} $, где m - произвольное целое

$$ I \approx \frac{h}{3} \left( f(a) + f(b) + 2 \sum_{i=1}^{m-1} f(a + 2ih) + 4 \sum_{i=1}^m f(a + (2i - 1) h) \right) $$

#### Правило Рунге оценки погрешности для формулы Симпсона

Пусть

$ I(h) $ - значение интеграла, вычисленного с шагом h = $ \frac{b - a}{2m} $

$ I(\frac{h}{2}) $ - значение интеграла, вычисленное с шагом $ \frac{h}{2} $ (т.е. с числом узлов 4m)

Тогда для оценки точности последнего вычисления используется величина

$$ R = \left| \frac{I(\frac{h}{2}) - I(h)}{2^4 - 1} \right| $$

#### Алгоритм автоматического выбора шага интегрирования

1. На первом шаге выбирается произвольное значения n — количество узлов интегрирования.
2. Далее вычисляются:

$ I(h) $ - значение интеграла, вычисленного с шагом h = $ \frac{b - a}{2m} $

$ I(\frac{h}{2}) $ - значение интеграла, вычисленное с шагом $ \frac{h}{2} $ (т.е. с числом узлов 4m)

3. Определяется погрешность по правилу Рунге. Если ее величина меньше заданного уровня погрешности $ \epsilon $, вычисления заканчиваются. За приближенное значение интеграла принимается $ I(\frac{h}{2}) $, либо проводится еще одно уточнение с помощью экстраполяции по Ричардсону. 

В противном случае вычисления повторяются с шага 2.

#### Замечания

1. Алгоритм должен предусматривать прекращение вычислений и выдачу последнего результата и соответствующего сообщения, если за некоторое, заранее выбранное, максимальное количество итераций не будет достигнут требуемый уровень погрешности.
2. Уровень адаптации алгоритма к поведению подынтегральной функции можно повысить, если на каждом частичном отрезке вычислять оценку погрешности по Рунге и далее уменьшать только тот отрезок, где эта погрешность максимальная. 

#### Экстраполяция по Ричардсону 

Для повышения точности вычислений применяют формулу: 
 
$$ I = \frac{2^4 I(\frac{h}{2}) - I(h)}{2^4 - 1} $$

$$ I \approx \frac{h}{3} \left( f(a) + f(b) + 2 \sum_{i=1}^{m-1} f(a + 2ih) + 4 \sum_{i=1}^m f(a + (2i - 1) h) \right) $$

In [79]:
def simpson_rule(f, a, b, m):
    h = (b - a) / (2 * m)
    result = f(a) + f(b)
    
    for i in range(1, m - 1):
        result += 2 * f(a + 2 * i * h)

    for i in range(1, m):
        result += 4 * f(a + (2 * i - 1) * h)

    result *= h / 3
    return result

def adaptive_simpson(f, a, b, eps, max_iter=100):
    n = 2  # начальное количество узлов интегрирования
    iter_count = 0

    while iter_count < max_iter:
        h = (b - a) / (2 * n)

        I_h = simpson_rule(f, a, b, n)
        I_h_over_2 = simpson_rule(f, a, b, 2 * n)

        runge_estimate = abs((I_h_over_2 - I_h) / (2**4 - 1))

        if runge_estimate < eps:
            # return I_h_over_2
            return (2**4 * I_h_over_2 - I_h) / (2**4 - 1)
        else:
            n *= 2
            iter_count += 1

    print("Максимальное количество итераций достигнуто. Результат может быть неточным.") 
    return I_h_over_2  # возвращаем последний результат после max_iter итераций


In [80]:
simpson_integral = adaptive_simpson(f, a, b, eps)

print(simpson_integral)

12.070230329686773


---

## Контрольные вопросы:

3.	Опишите способ численного дифференцирования, основанный на интерполяции алгебраическими многочленами.

Численное дифференцирование на основе интерполяции алгебраическими многочленами осуществляется путем аппроксимации функции многочленом и последующего вычисления производной этого многочлена.

Метод применения алгебраической интерполяции для вычисления производных состоит в получении интерполяционного многочлена по заданным значениям функции. Значение производной функции принимается равным значению производной от этого многочлена. Недостаток метода состоит в том, что хороший результат интерполяции не гарантирует хорошего приближения производной.

Если P(x) — интерполяционный многочлен по заданным узлам и r(x, f) — погрешность интерполяции, то:

$$ 
f(x) = P(x) + r(x, f)
\\
f^{'} (x) = P^{'} (x) + r^{'} (x, f)
\\
f^{'} = P^{'} (x) + R(x, f), \ R(x, f) = r^{'} (x, f)
$$

Погрешность этой формулы – производная от погрешности интерполяции, поэтому хороший результат интерполяции не гарантирует хорошего приближения производной.

---

6.	Опишите способ получения квадратурных формул интерполяционного типа (интерполяционных квадратурных формул) и оценок их погрешности. 

Получение интерполяционных квадратурных формул основано на замене подынтегральной функции интерполяционным полиномом по заранее выбранным узлам. За приближенное значение интеграла принимается интеграл от этого полинома.

$$
f(x) = P(x) + r(x, f)
\\
\int_a^b f(x) dx = \int_a^b P(x) dx + \int_a^b r(x, f) dx
\\
\int_a^b f(x) dx = \int_a^b P(x) dx + R(f), \ R(F) = \int_a^b r(x, f) dx
$$

Погрешность этой формулы – определенный интеграл на отрезке интерполяции от функции, которая определяет погрешность интерполяции, поэтому хороший результат интерполяции гарантирует хорошее приближение интеграла.

---

7.	Пусть для таблично заданной функции F(x), единственным образом получена функция G(x), интерполирующая F(x): 

$$ F(x) = G(x) + r(x, F) $$

В каком случае можно утверждать, что если остаток (т.е. $ r(x, F) $) этой интерполяции мал на интервале [a, b], то на этом интервале также будет мала погрешность приближения:

1. $ F^{'} (x) = G^{'} (x) + R(x, F) $, $ R(x, F) = r^{'} (x, F) $
2. $ \int_a^b F(x) dx = \int_a^b G(x) dx + R(F) $, $ R(F) = \int_a^b r(x, F) dx $

Поясните ответ.

1. $ F^{'} (x) = G^{'} (x) + R(x, F) $, $ R(x, F) = r^{'} (x, F) $

$ R(x, F) $ - это погрешность в производной. Таким образом, если остаток $ r(x, F) $ мал, то и его производная $ r^{'} (x, F) $ также мала, и погрешность приближения $ F^{'} (x, F) $  будет мала.

2. $ \int_a^b F(x) dx = \int_a^b G(x) dx + R(F) $, $ R(F) = \int_a^b r(x, F) dx $

$ R(F) $ - это погрешность в интеграле. Таким образом, если остаток $ r(x, F) $ мал, то и его интеграл $ R(F) $ также будет мал, и погрешность приближения $ \int_a^b F(x) dx $ будет мала.